In [190]:
import pickle
import re
import requests, bs4
import numpy as np
import pandas as pd
import os

In [208]:
file = 'WNBA16.csv'

In [209]:
db = pd.read_csv(f'{file}')

In [210]:
team_abbrevs = {
    'Atlanta Dream': 'ATL',
    'Washington Mystics': 'WAS',
    'Connecticut Sun': 'CON',
    'Chicago Sky': 'CHI',
    'New York Liberty': 'NYL',
    'Indiana Fever': 'IND',
    'Seattle Storm': 'SEA',
    'Phoenix Mercury': 'PHO',
    'L.A. Sparks':'LAS',
    'Minnesota Lynx': 'MIN',
    'Dallas Wings': 'DAL',
    'Las Vegas Aces': 'LVA'
}

team_abbrevs17 = {
    'Atlanta Dream': 'ATL',
    'Washington Mystics': 'WAS',
    'Connecticut Sun': 'CON',
    'Chicago Sky': 'CHI',
    'New York Liberty': 'NYL',
    'Indiana Fever': 'IND',
    'Seattle Storm': 'SEA',
    'Phoenix Mercury': 'PHO',
    'Los Angeles Sparks':'LAS',
    'Minnesota Lynx': 'MIN',
    'Dallas Wings': 'DAL',
    'San Antonio Stars': 'SAS'
}

team_abbrevs16 = {
    'Atlanta Dream': 'ATL',
    'Washington Mystics': 'WAS',
    'Connecticut Sun': 'CON',
    'Chicago Sky': 'CHI',
    'New York Liberty': 'NYL',
    'Indiana Fever': 'IND',
    'Seattle Storm': 'SEA',
    'Phoenix Mercury': 'PHO',
    'Los Angeles Sparks':'LAS',
    'Minnesota Lynx': 'MIN',
    'Dallas Wings': 'DAL',
    'San Antonio Stars': 'SAS'
}

In [123]:
def date_formatter(date):

    if len(str(date.month)) != 2:
        month = '0'+ str(date.month)
    else:
        month = str(date.month)
        
    if len(str(date.day)) != 2:
        day = '0' + str(date.day)
    else:
        day = str(date.day)
        
    newdate = str(date.year) + month + day
    return newdate

In [124]:
db['Date'] = pd.to_datetime(db['Date'])
db['Date'] = db['Date'].apply(date_formatter)

In [130]:
db['Date']

0      20170513
1      20170519
2      20170521
3      20170527
4      20170531
5      20170604
6      20170607
7      20170610
8      20170613
9      20170615
10     20170623
11     20170630
12     20170702
13     20170705
14     20170707
15     20170709
16     20170712
17     20170715
18     20170718
19     20170719
20     20170725
21     20170728
22     20170730
23     20170803
24     20170805
25     20170808
26     20170811
27     20170812
28     20170815
29     20170819
         ...   
378    20170528
379    20170531
380    20170603
381    20170606
382    20170610
383    20170615
384    20170618
385    20170621
386    20170623
387    20170625
388    20170630
389    20170705
390    20170707
391    20170712
392    20170716
393    20170718
394    20170720
395    20170725
396    20170728
397    20170730
398    20170801
399    20170804
400    20170805
401    20170810
402    20170812
403    20170818
404    20170822
405    20170825
406    20170901
407    20170902
Name: Date, Length: 408,

In [131]:
# Moneyline scrape

scraped_dates = []
moneydb = pd.DataFrame()

for date in db['Date']:
    
    #if date == 20160616:
     #   pass
    
    if date not in scraped_dates:
        print("Date: ", date)
        url = f'https://www.sportsbookreview.com/betting-odds/wnba-basketball/money-line/?date={date}'
        res = requests.get(url, 'html.parser')
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text, 'lxml')
        linesoup = soup.find('div', {'id':'bettingOddsGridContainer'})
        test_teams = linesoup.find_all('span', {'class':'_3O1Gx'})
        teams = [team_abbrevs[team.get_text()] for team in test_teams]
        test_lines = linesoup.find_all('span', {'class':'opener'})
        lines = [line.get_text() for line in test_lines if line.get_text().startswith('-') and len(line.get_text()) > 1 or line.get_text().startswith('+') and len(line.get_text()) > 1]
        print("teams:",len(teams),"lines",len(lines))
        
        if len(teams) != len(lines):
            pass
        else:
            moneylines = {'Team':teams, 'ML Odds': lines, 'Date':[date]*len(teams)}
        
            MLdb = pd.DataFrame.from_dict(moneylines)
            moneydb = pd.concat([moneydb, MLdb])
            
        scraped_dates.append(date)
        
# moneydb.to_csv('MLodds.csv')

Date:  20170513
teams: 6 lines 6
Date:  20170519
teams: 6 lines 6
Date:  20170521
teams: 4 lines 4
Date:  20170527
teams: 4 lines 4
Date:  20170531
teams: 4 lines 4
Date:  20170604
teams: 4 lines 4
Date:  20170607
teams: 4 lines 4
Date:  20170610
teams: 6 lines 6
Date:  20170613
teams: 4 lines 4
Date:  20170615
teams: 4 lines 4
Date:  20170623
teams: 10 lines 10
Date:  20170630
teams: 6 lines 6
Date:  20170702
teams: 4 lines 4
Date:  20170705
teams: 6 lines 6
Date:  20170707
teams: 4 lines 4
Date:  20170709
teams: 4 lines 4
Date:  20170712


KeyboardInterrupt: 

In [10]:
moneydb.to_csv('moneydb19.csv', index = False)

In [11]:
db['Odds'] = np.zeros(len(db['Team']))

In [12]:
for i, date in enumerate(db['Date']):
    
    if date in list(moneydb['Date']):
        
        db['Odds'][i] = moneydb[(moneydb['Date'] == date) & (moneydb['Team'] == db['Team'][i])]['ML Odds']
        print("Team:", db['Team'][i], "Odds:", moneydb[(moneydb['Date'] == date) & (moneydb['Team'] == db['Team'][i])]['ML Odds'])
        
    else:
        db['Odds'][i] = np.nan

/Users/ahelgeso/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Team: ATL Odds: 1    -380
Name: ML Odds, dtype: object
Team: ATL Odds: 1    -265
Name: ML Odds, dtype: object
Team: ATL Odds: 2    -375
Name: ML Odds, dtype: object
Team: ATL Odds: 1    +130
Name: ML Odds, dtype: object
Team: ATL Odds: 3    +185
Name: ML Odds, dtype: object
Team: ATL Odds: 2    -150
Name: ML Odds, dtype: object
Team: ATL Odds: 1    -125
Name: ML Odds, dtype: object
Team: ATL Odds: 0    +650
Name: ML Odds, dtype: object
Team: ATL Odds: 1    +325
Name: ML Odds, dtype: object
Team: ATL Odds: 1    -155
Name: ML Odds, dtype: object
Team: ATL Odds: 2    +225
Name: ML Odds, dtype: object
Team: ATL Odds: 2    +220
Name: ML Odds, dtype: object
Team: ATL Odds: 4    +270
Name: ML Odds, dtype: object
Team: ATL Odds: 1    +180
Name: ML Odds, dtype: object
Team: ATL Odds: 3    -103
Name: ML Odds, dtype: object
Team: ATL Odds: 1    +158
Name: ML Odds, dtype: object
Team: WAS Odds: 2    +100
Name: ML Odds, dtype: object


/Users/ahelgeso/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Team: WAS Odds: 3    +285
Name: ML Odds, dtype: object
Team: WAS Odds: 1    -700
Name: ML Odds, dtype: object
Team: WAS Odds: 2    -440
Name: ML Odds, dtype: object
Team: WAS Odds: 1    -1450
Name: ML Odds, dtype: object
Team: WAS Odds: 2    +175
Name: ML Odds, dtype: object
Team: WAS Odds: 1    -450
Name: ML Odds, dtype: object
Team: WAS Odds: 0    +135
Name: ML Odds, dtype: object
Team: WAS Odds: 2    +170
Name: ML Odds, dtype: object
Team: WAS Odds: 0    -400
Name: ML Odds, dtype: object
Team: WAS Odds: 0    -230
Name: ML Odds, dtype: object
Team: WAS Odds: 1    -295
Name: ML Odds, dtype: object
Team: WAS Odds: 2    -160
Name: ML Odds, dtype: object
Team: WAS Odds: 3    -900
Name: ML Odds, dtype: object
Team: WAS Odds: 1    -235
Name: ML Odds, dtype: object
Team: CON Odds: 3    -120
Name: ML Odds, dtype: object
Team: CON Odds: 1    -550
Name: ML Odds, dtype: object
Team: CON Odds: 4    +115
Name: ML Odds, dtype: object
Team: CON Odds: 0    +215
Name: ML Odds, dtype: object
Team: CON

In [14]:
db.to_csv(f'{file}', index = False)

## Below is for troubleshooting

In [9]:
url = 'https://www.sportsbookreview.com/betting-odds/wnba-basketball/?date=20180703'

In [10]:
res = requests.get(url, 'html.parser')
soup = bs4.BeautifulSoup(res.text, 'lxml')
linesoup = soup.find('div', {'id':'bettingOddsGridContainer'})
games = soup.find('div', {'class':'_1eZfC'})

In [19]:
pd.read_html(str(games), flavor = 'bs4', attrs = {'class':'_1eZfC'})

ValueError: No tables found

In [211]:
spreadfile = '16Spreads.csv'

In [212]:
spreaddb = pd.read_csv(f'{spreadfile}')

### Scraping lines remnants from NBA scraper

In [132]:
# Point spread scrape
scraped_dates = []
spreaddb = pd.DataFrame()

for date in db['Date']:
    if date not in scraped_dates:
        print("Date: ", date)
        url = f'https://www.sportsbookreview.com/betting-odds/wnba-basketball/pointspread/?date={date}'
        res = requests.get(url, 'html.parser')
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text, 'lxml')
        linesoup = soup.find('div', {'id':'bettingOddsGridContainer'})
        test_teams = linesoup.find_all('span', {'class':'_3O1Gx'})
        teams = [team_abbrevs[team.get_text()] for team in test_teams]
        test_lines = linesoup.find_all('span', {'class':'_3Nv_7 opener'})
        lines = [line.get_text() for line in test_lines if line.get_text().startswith('-') or line.get_text().startswith('+') or line.get_text().startswith('P')]
        print("teams:",len(teams),"lines",len(lines))
        
        if len(teams) != len(lines):
            lines = [np.nan for team in teams]
            spread = {'Team':teams, 'Spread': lines, 'Date':[date]*len(lines)}
            spreads = pd.DataFrame.from_dict(spread)
            spreaddb = pd.concat([spreaddb, spreads])
            scraped_dates.append(date)
            
        else:
            spread = {'Team':teams, 'Spread': lines, 'Date':[date]*len(lines)}
            spreads = pd.DataFrame.from_dict(spread)
            spreaddb = pd.concat([spreaddb, spreads])
            scraped_dates.append(date)

# spreaddb.to_csv('19Spreads.csv', index = False)

Date:  20170513
teams: 6 lines 6
Date:  20170519
teams: 6 lines 6
Date:  20170521
teams: 4 lines 4
Date:  20170527
teams: 4 lines 4
Date:  20170531
teams: 4 lines 4
Date:  20170604
teams: 4 lines 4
Date:  20170607
teams: 4 lines 4
Date:  20170610
teams: 6 lines 6
Date:  20170613
teams: 4 lines 4
Date:  20170615
teams: 4 lines 4
Date:  20170623
teams: 10 lines 10
Date:  20170630
teams: 6 lines 6
Date:  20170702
teams: 4 lines 4
Date:  20170705
teams: 6 lines 6
Date:  20170707
teams: 4 lines 4
Date:  20170709
teams: 4 lines 4
Date:  20170712
teams: 8 lines 8
Date:  20170715
teams: 2 lines 2
Date:  20170718
teams: 4 lines 4
Date:  20170719
teams: 8 lines 8
Date:  20170725
teams: 12 lines 12
Date:  20170728
teams: 12 lines 12
Date:  20170730
teams: 12 lines 12
Date:  20170803
teams: 2 lines 2
Date:  20170805
teams: 4 lines 4
Date:  20170808
teams: 6 lines 6
Date:  20170811
teams: 4 lines 4
Date:  20170812
teams: 8 lines 8
Date:  20170815
teams: 2 lines 2
Date:  20170819
teams: 2 lines 2
Da

In [220]:
db['Line'] = np.zeros(len(db['Team']))

In [221]:
spreaddb = spreaddb.replace('LVA', 'SAS')

In [222]:
#db['Odds'][0] = moneydb[(moneydb['Date'] == date) & (moneydb['Team'] == db['Team'][0])]['ML Odds']
#db['Line'][0] = spreaddb[(spreaddb['Date'] == date) & (spreaddb['Team'] == db['Team'][0])]['Spread']

#moneydb.reset_index(drop = True, inplace = True)
spreaddb.reset_index(drop = True, inplace = True)



In [223]:
for i, spread in enumerate(spreaddb['Spread']):
    if type(spread) == str:
        spreaddb['Spread'][i] = re.sub("½", ".5", spread)
    else:
        pass
    
spreaddb['Spread'] = spreaddb['Spread'].replace('PK',0)

In [224]:
#spreaddb['Spread'] = spreaddb['Spread'].replace('PK',0)
spreaddb['Spread'] = spreaddb['Spread'].apply(lambda x: float(x))

In [225]:
spreaddb['Date'].apply(lambda x: int(x))
db.Date = db['Date'].apply(lambda x: int(x))
#moneydb['Date'].apply(lambda x: int(x))
#moneydb['ML Odds'] = moneydb['ML Odds'].apply(lambda x: int(x))

In [226]:
for i, date in enumerate(db['Date']):
    
    mask = (spreaddb['Date'] == date) & (spreaddb['Team'] == db['Team'][i])
    if any(mask):
        db['Line'][i] = spreaddb[mask]['Spread']
        print("Team:", db['Team'][i], "Spread:", spreaddb[(spreaddb['Date'] == date) & (spreaddb['Team'] == db['Team'][i])]['Spread'])
    else:
        db['Line'][i] = np.nan

/Users/ahelgeso/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ahelgeso/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Team: ATL Spread: 2    5.5
Name: Spread, dtype: float64
Team: ATL Spread: 7    4.0
Name: Spread, dtype: float64
Team: ATL Spread: 10    8.0
Name: Spread, dtype: float64
Team: ATL Spread: 18    3.5
Name: Spread, dtype: float64
Team: ATL Spread: 21   -3.0
Name: Spread, dtype: float64
Team: ATL Spread: 26   -5.5
Name: Spread, dtype: float64
Team: ATL Spread: 37   -11.0
Name: Spread, dtype: float64
Team: ATL Spread: 45    4.5
Name: Spread, dtype: float64
Team: ATL Spread: 47   -10.5
Name: Spread, dtype: float64
Team: ATL Spread: 53   -1.5
Name: Spread, dtype: float64
Team: ATL Spread: 58    2.5
Name: Spread, dtype: float64
Team: ATL Spread: 63    1.5
Name: Spread, dtype: float64
Team: ATL Spread: 66   -4.0
Name: Spread, dtype: float64
Team: ATL Spread: 70    2.5
Name: Spread, dtype: float64
Team: ATL Spread: 74    12.5
Name: Spread, dtype: float64
Team: ATL Spread: 81    1.5
Name: Spread, dtype: float64
Team: ATL Spread: 83   -3.5
Name: Spread, dtype: float64
Team: ATL Spread: 89   -2.5
Na

In [203]:
#db.to_csv(f'{file}', index = False)

In [227]:
db['Cover'] = np.zeros(len(db['Team']))

In [ ]:
## for i in range(len(db['Cover'])):
    cover = db['Team Score'][i] + db['Line'][i] - db['Opp Score'][i]
    if cover > 0:
        db['Cover'][i] = 1
    elif cover < 0:
        db['Cover'][i] = -1
    else:
        db['Cover'][i] = 0
        

In [229]:
db.to_csv(f'{file}', index = False)

In [230]:
spreaddb.to_csv(f'{spreadfile}', index = False)

In [176]:
spreaddb[(spreaddb['Date'] == db['Date'][0]) & (spreaddb['Team'] == db['Team'][0])]

,Team,Spread,Date


In [186]:
mask = (spreaddb['Date'] == db['Date'][1]) & (spreaddb['Team'] == db['Team'][1])

In [189]:
db

,Team,Opp,Game Num,Date,Team Score,Opp Score,Result,Location,TSP,TOV,...,Opp R4TSP,Opp R4TOV,Opp R4ORB,Opp R4FTR,Opp R4Poss,Opp R4OEff,Opp R4DEff,Odds,0,Line
0,ATL,SAS,1,20160514,73,63,1,0,0.473288,0.162831,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ATL,IND,2,20160520,85,94,0,0,0.552810,0.214548,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.0,0.327869,5.5
2,ATL,CHI,3,20160522,87,81,1,1,0.502078,0.054561,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160.0,0.384615,4.0
3,ATL,NYL,4,20160524,85,79,1,0,0.449831,0.144823,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,290.0,0.256410,8.0
4,ATL,DAL,5,20160527,102,93,1,0,0.617733,0.126904,...,0.510871,0.131757,0.312669,0.326065,77.7216,106.138819,100.370394,165.0,0.377358,3.5
5,ATL,IND,6,20160529,85,76,1,1,0.517788,0.108601,...,0.521178,0.147577,0.256467,0.287726,82.3488,102.306886,102.170606,-200.0,0.666667,-3.0
6,ATL,CON,7,20160603,83,77,1,0,0.509578,0.164204,...,0.498652,0.156795,0.277003,0.220787,82.8384,96.591285,106.941612,-225.0,0.692308,-5.5
7,ATL,WAS,8,20160605,79,86,0,1,0.500761,0.092081,...,0.539686,0.153512,0.274567,0.307978,76.5600,105.611311,110.896934,-600.0,0.857143,-11.0
8,ATL,MIN,9,20160610,78,110,0,1,0.479823,0.181305,...,0.525783,0.138970,0.199488,0.221424,79.0368,101.779262,89.755659,190.0,0.344828,4.5
9,ATL,CON,10,20160612,93,87,1,1,0.572378,0.137946,...,0.522644,0.173877,0.232813,0.199077,81.9456,98.289117,105.874250,-650.0,0.866667,-10.5
